In [1]:
import math
import time
import pandas as pd
import numpy as np
import numpy.linalg as la

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor

In [2]:
def load_data(path):
    df = pd.read_csv(path)
    data = df.drop(columns=['Unnamed: 0'])
    
    return data

In [3]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate)
    train_data = data[0:train_size]
    test_data = data[train_size:time_len]
    
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len):
        a = train_data[i: i + seq_len + pre_len]
        trainX.append(a[0 : seq_len])
        trainY.append(a[seq_len : seq_len + pre_len])
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
        
    return trainX, trainY, testX, testY

In [4]:
def getTestY(data):
    tmp_scaler = StandardScaler()
    tmp_data = tmp_scaler.fit_transform(data)
    tmp_data = tmp_scaler.inverse_transform(tmp_data)
    _, _, _, testY = preprocess_data(tmp_data, data.shape[0], train_rate, seq_len, pre_len)
    testY = np.array(testY)
    testY = np.reshape(testY, [-1, pre_len])
    
    return testY

In [5]:
 def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    mape = mean_absolute_percentage_error(a, b)
    F_norm = la.norm(a-b)/la.norm(a)
    
    return rmse, mae, mape, 1-F_norm

In [6]:
def predictSVR(data, testY):
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    
    rmses, maes, mapes, accs = [], [], [], []
    for i in range(data.shape[1]):
        print('Node', i)
        start = time.time()
        a = data[:, i]
        aX, aY, tX, tY = preprocess_data(a, data.shape[0], train_rate, seq_len, pre_len)
        
        aX = np.array(aX)
        aX = np.reshape(aX, [-1, seq_len])
        aY = np.array(aY)
        aY = np.reshape(aY, [-1, pre_len])

        tX = np.array(tX)
        tX = np.reshape(tX, [-1, seq_len])
        tY = np.array(tY)
        tY = np.reshape(tY, [-1, pre_len])
        
        print('(', aX.shape, aY.shape, tX.shape, tY.shape, round(aX.mean(),3), ')')
        reg = MultiOutputRegressor(SVR(kernel='linear'))
        reg.fit(aX, aY)
        pred = reg.predict(tX)
        
        mean = scaler.mean_[i]
        std = np.sqrt(scaler.var_[i])
        pred = pred*std + mean
        tY = tY*std + mean
        if i==data.shape[1]-1: tY = testY[tY.shape[0]*i:]
        else: tY = testY[tY.shape[0] * i:tY.shape[0] * (i+1)]
        
        rmse, mae, mape, acc = evaluation(tY, pred)
        rmses.append(rmse)
        maes.append(mae)
        mapes.append(mape)
        accs.append(acc)
        
        print('(', rmse, mae, mape, acc, time.time() - start)
    
    print('RMSE: ' + str(np.mean(rmses)) + ', MAE: ' + str(np.mean(maes)) + ', MAPE: ' + str(np.mean(mapes)) + ', ACC: ' + str(np.mean(accs)))

In [7]:
train_rate = 0.8
seq_len = 12
pre_len = 6

In [8]:
path_0 = '../Data/Gangnam/speed_gangnam_0.csv'
path_5 = '../Data/Gangnam/speed_gangnam_5.csv'
path_10 = '../Data/Gangnam/speed_gangnam_10.csv'
path_20 = '../Data/Gangnam/speed_gangnam_20.csv'

In [9]:
masterData = load_data(path_0)
print(masterData.shape, (masterData == 0).sum().sum())
testY = getTestY(masterData)
testY.shape, (testY == 0).sum().sum()

(2880, 506) 0


((282348, 6), 0)

### Unobserved Nodes = 10%

In [10]:
data = load_data(path_10)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(2880, 506) 144000
Node 0
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.004 )
( 10.751276746671783 7.618165517823226 0.24987702501566178 0.663807065587046 2.6149728298187256
Node 1
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.032 )
( 13.513944003597526 9.983707317518416 0.31859579526579085 0.5819834893230527 2.116342782974243
Node 2
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.095 )
( 12.38422530722103 9.593065952740226 0.3445397670813946 0.6169693301328221 2.3896422386169434
Node 3
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 32.524998489811594 30.85213261648745 1.0 0.0 0.007974863052368164
Node 4
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.006 )
( 16.35642354792104 12.753406233065858 0.37963257523328253 0.4982359402774631 2.651886463165283
Node 5
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.02 )
( 13.517687473790497 9.821299680585442 0.283619375793492 0.5873567308754998 3.010952949523926
Node 6
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 32.91952895182124 31.169704301075257 1

( 13.276367038547695 10.672128125611598 0.4158717849409741 0.5998776550514461 3.2213902473449707
Node 57
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.007 )
( 11.798079923696465 8.711305523855204 0.2985836650600638 0.6430399222017549 3.365006923675537
Node 58
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.037 )
( 15.657903699017638 13.260032052494957 0.5291498995634356 0.5262812602865952 2.844398260116577
Node 59
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.0 )
( 12.072220544575377 9.347112127438285 0.35534926687828944 0.6343862972002026 3.299182653427124
Node 60
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.052 )
( 11.377578280935872 8.806733806196677 0.34096874755584405 0.6550099500995592 2.9939987659454346
Node 61
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.062 )
( 16.133563865087854 12.680488193093957 0.3688930800094676 0.5131722500419786 2.769596576690674
Node 62
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.003 )
( 11.312106395014332 8.564991649802565 0.3254621783304297 0.657420892428

( 10.261583427532901 7.943644070965159 0.3886867646946514 0.6187408968646124 3.9634063243865967
Node 111
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.017 )
( 9.492549096663089 6.896255775348847 0.2984211081726486 0.6455975200527274 3.107696533203125
Node 112
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.025 )
( 9.92946500131185 7.4951421843513755 0.3557163338006027 0.6281201142647133 3.1555655002593994
Node 113
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.002 )
( 10.747176905386894 8.218409776073551 0.35949959714619445 0.5969375507089676 3.115673542022705
Node 114
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.039 )
( 9.150800318586997 6.685187385645096 0.32952389502370927 0.6552831333878761 4.243656635284424
Node 115
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.016 )
( 10.214708670967186 7.709952172150579 0.36667684106611514 0.6155474039470799 3.996319532394409
Node 116
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.049 )
( 11.764192459333211 9.65063851458651 0.5184037595591694 0.558064915088134

( 11.961652687593192 9.466738669608318 0.5033833959964632 0.5460082592142459 2.8074517250061035
Node 166
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.062 )
( 18.110345305770686 15.059398193477515 0.8145922253752537 0.3042042746844028 2.1732351779937744
Node 167
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.058 )
( 17.70351760368119 14.324514985296739 0.7764914089515006 0.32686128802298864 2.4095585346221924
Node 168
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.005 )
( 28.91913418078668 27.229710352841355 1.4350133034419372 -0.09488405055187643 3.1785011291503906
Node 169
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.008 )
( 16.204665893798605 13.352616217087585 0.6583053386050247 0.3831977966682353 1.9208297729492188
Node 170
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.021 )
( 10.445026862692613 7.70215761305671 0.35850177827751667 0.6030671631021091 3.100712537765503
Node 171
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.005 )
( 12.842669599275942 10.50716436776253 0.5658606270106489 0.51018

( 12.200354938055053 9.8624972400882 0.44494502145482445 0.5797262039036025 3.1207048892974854
Node 220
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.007 )
( 12.851775799267365 10.236020191762366 0.4568799523396801 0.5616372426507622 2.81050968170166
Node 221
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.015 )
( 10.43998150598834 8.056413860627897 0.3574390643841289 0.6430666119003161 3.292135238647461
Node 222
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.016 )
( 10.13989663498449 7.176995783297609 0.27518174322497346 0.6525130459751756 3.421121597290039
Node 223
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.024 )
( 10.791397954325854 7.950700488530987 0.31437194191997875 0.6309957312966066 3.113675832748413
Node 224
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 29.14171580910578 27.51049581839906 1.0 0.0 0.007978200912475586
Node 225
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.032 )
( 10.58356505523481 7.759800334194625 0.31163126631797905 0.6390051823968252 3.247321128845215
Node 226

( 13.807349954717358 10.193439289310964 0.30323381733527743 0.5932647637351174 3.3879458904266357
Node 274
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.0 )
( 16.35655004335226 12.72689313545631 0.36394187855574206 0.5176356180885207 2.8683347702026367
Node 275
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.024 )
( 16.108961823273464 12.391186395644917 0.35000840129328087 0.528820180778318 3.0588650703430176
Node 276
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 33.95903111757525 32.09511350059738 1.0 0.0 0.007981300354003906
Node 277
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.13 )
( 18.149714772033718 14.631300455735015 0.4108374982985252 0.4682456863756209 3.2512667179107666
Node 278
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.051 )
( 14.735068124901403 11.027907695893035 0.31483171164810947 0.5682945231268615 3.5205914974212646
Node 279
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.043 )
( 15.766028118875116 12.079440919830487 0.34440786246116356 0.5388021893013855 3.465738773345947

( 16.872563504057336 14.03715775661768 0.5952130976210849 0.49088559018945543 4.186761140823364
Node 328
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 33.06007268419278 31.144059139784943 1.0 0.0 0.010970592498779297
Node 329
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.044 )
( 12.491642523385071 9.532862579086057 0.3465573169161802 0.6191118157223707 3.5804314613342285
Node 330
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 32.75044960201022 30.919071087216242 1.0 0.0 0.008026123046875
Node 331
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.008 )
( 12.315961936248401 9.83705028365276 0.39897403977312534 0.6222752114216596 2.783513069152832
Node 332
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.026 )
( 12.062291843407529 9.288174489156232 0.3464553296036738 0.6299218958352372 3.0668342113494873
Node 333
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.096 )
( 13.075207801680008 10.835393232755107 0.45315737040539855 0.5967711654161374 3.4547786712646484
Node 334
( (2286, 12) (2286, 6) (558

( 11.338542436270792 8.628112059518106 0.4211809988765771 0.5708648590290175 3.114675521850586
Node 382
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.035 )
( 13.033736888175236 10.099246069862458 0.463430689659571 0.5059588962089923 2.7436678409576416
Node 383
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.031 )
( 10.397014555992682 7.7805633570986545 0.392899632413981 0.6044100658770073 3.217400550842285
Node 384
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.054 )
( 10.511801377188188 7.775533473594186 0.38251714531659714 0.5979948277658371 2.8803021907806396
Node 385
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 26.07416886764735 24.147801672640384 1.0 0.0 0.007978677749633789
Node 386
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.001 )
( 10.936033069946049 8.841316691048089 0.4881283430823758 0.5791142939147559 3.8636720180511475
Node 387
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.031 )
( 12.049511130466154 9.790110103186336 0.5334821734247505 0.5373104734949776 3.038879156112671
Node 

( 10.02150953128456 7.261870254332165 0.35455316917315893 0.6029947460558182 2.8154754638671875
Node 436
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.012 )
( 11.400632042776204 8.194941041894513 0.34778167850949443 0.5494699258412015 2.789545774459839
Node 437
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.018 )
( 10.969614829874342 8.218725656615245 0.43477927743760897 0.5662541100813558 2.665874481201172
Node 438
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.02 )
( 10.169182928590963 7.876543714937579 0.45430067945155234 0.5966422431292213 3.566467046737671
Node 439
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.023 )
( 11.498811608477844 9.382533833724386 0.5690345071157238 0.5462232472521431 3.783888339996338
Node 440
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.048 )
( 15.113206506951185 12.985264371394178 0.7819395930837718 0.4001212256026102 3.485960006713867
Node 441
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.015 )
( 11.62855079103698 9.727372915191259 0.5845909173727041 0.53892850998

( 17.519426010639965 15.784728028642853 0.7323977334402373 0.404560899117972 2.8195066452026367
Node 490
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.026 )
( 15.060384353980721 13.253489222419821 0.6244468084277773 0.485193218923311 4.043154001235962
Node 491
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.025 )
( 10.39356296856954 7.4555157020007075 0.28688665511084777 0.6451050977240865 2.9341559410095215
Node 492
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.012 )
( 12.305621506991612 8.987747373434722 0.310938315296078 0.5792731214416814 3.226377487182617
Node 493
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 29.471169971148743 27.85717144563918 1.0 0.0 0.010971784591674805
Node 494
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.008 )
( 11.844962891997575 8.644321534679142 0.29847987738402604 0.6034171696078061 3.320099353790283
Node 495
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 10.516212010603617 7.698698766783228 0.2904566851192736 0.6492244875986566 3.1206774711608887
Node 

### Unobserved Nodes = 20%

In [11]:
data = load_data(path_20)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(2880, 506) 290880
Node 0
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.004 )
( 10.751276746671783 7.618165517823226 0.24987702501566178 0.663807065587046 3.116684675216675
Node 1
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.032 )
( 13.513944003597526 9.983707317518416 0.31859579526579085 0.5819834893230527 2.6428918838500977
Node 2
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.095 )
( 12.38422530722103 9.593065952740226 0.3445397670813946 0.6169693301328221 2.96895432472229
Node 3
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 32.524998489811594 30.85213261648745 1.0 0.0 0.012963294982910156
Node 4
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.006 )
( 16.35642354792104 12.753406233065858 0.37963257523328253 0.4982359402774631 3.178588390350342
Node 5
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.02 )
( 13.517687473790497 9.821299680585442 0.283619375793492 0.5873567308754998 3.65193772315979
Node 6
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.012 )
( 12.04417941026537 8.423232392510839 0.2

( 10.672558429607278 7.685206048928667 0.25891402037899675 0.6774547621202135 3.6464552879333496
Node 56
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.003 )
( 13.276367038547695 10.672128125611598 0.4158717849409741 0.5998776550514461 2.753640651702881
Node 57
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.007 )
( 11.798079923696465 8.711305523855204 0.2985836650600638 0.6430399222017549 2.979086399078369
Node 58
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.037 )
( 15.657903699017638 13.260032052494957 0.5291498995634356 0.5262812602865952 2.4663596153259277
Node 59
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.0 )
( 12.072220544575377 9.347112127438285 0.35534926687828944 0.6343862972002026 2.8673696517944336
Node 60
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 32.9794389264829 31.284280167264047 1.0 0.0 0.007976531982421875
Node 61
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.062 )
( 16.133563865087854 12.680488193093957 0.3688930800094676 0.5131722500419786 2.3916094303131104
Node 62

( 9.492549096663089 6.896255775348847 0.2984211081726486 0.6455975200527274 2.8439483642578125
Node 112
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 26.70073155927526 25.174280167264044 1.0 0.0 0.00994110107421875
Node 113
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.002 )
( 10.747176905386894 8.218409776073551 0.35949959714619445 0.5969375507089676 2.8015427589416504
Node 114
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.039 )
( 9.150800318586997 6.685187385645096 0.32952389502370927 0.6552831333878761 3.5884110927581787
Node 115
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.016 )
( 10.214708670967186 7.709952172150579 0.36667684106611514 0.6155474039470799 3.456761360168457
Node 116
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.049 )
( 11.764192459333211 9.65063851458651 0.5184037595591694 0.5580649150881343 2.865344285964966
Node 117
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.042 )
( 20.8998971178594 19.396824905192446 1.03339921954663 0.21336146071522277 2.6409077644348145
Node 118

( 17.70351760368119 14.324514985296739 0.7764914089515006 0.32686128802298864 2.2310190200805664
Node 168
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.005 )
( 28.91913418078668 27.229710352841355 1.4350133034419372 -0.09488405055187643 3.1874494552612305
Node 169
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.008 )
( 16.204665893798605 13.352616217087585 0.6583053386050247 0.3831977966682353 1.8467626571655273
Node 170
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.021 )
( 10.445026862692613 7.70215761305671 0.35850177827751667 0.6030671631021091 2.707927942276001
Node 171
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.005 )
( 12.842669599275942 10.50716436776253 0.5658606270106489 0.5101860713914417 3.020940065383911
Node 172
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.017 )
( 13.712321012636465 10.95151468577037 0.5028743522697852 0.4721085667759264 2.624941349029541
Node 173
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.03 )
( 11.228744279470305 8.82535275159034 0.4628958469027946 0.5687534776

( 10.43998150598834 8.056413860627897 0.3574390643841289 0.6430666119003161 3.0508463382720947
Node 222
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.016 )
( 10.13989663498449 7.176995783297609 0.27518174322497346 0.6525130459751756 3.1047065258026123
Node 223
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.024 )
( 10.791397954325854 7.950700488530987 0.31437194191997875 0.6309957312966066 3.1087687015533447
Node 224
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 29.14171580910578 27.51049581839906 1.0 0.0 0.007996082305908203
Node 225
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 29.317775599950146 27.613076463560343 1.0 0.0 0.0069828033447265625
Node 226
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.008 )
( 11.021227538534536 7.926742587548979 0.28976064011318015 0.6218709156030355 3.357548713684082
Node 227
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 29.079992866667762 27.40394265232976 1.0 0.0 0.010970830917358398
Node 228
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.005 )
( 1

( 15.766028118875116 12.079440919830487 0.34440786246116356 0.5388021893013855 3.4429452419281006
Node 280
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.002 )
( 13.407157285701322 9.837064454178085 0.30818418611490267 0.6114283026909532 3.624281406402588
Node 281
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.05 )
( 15.820664838403317 11.9989997127225 0.34214169222543284 0.54237858210414 3.4168999195098877
Node 282
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.001 )
( 15.77731182047858 12.080097870629443 0.3755945054510695 0.5439187479919331 2.8403780460357666
Node 283
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.007 )
( 14.304663037370112 10.801789093850596 0.35244204278331503 0.5879543337397011 2.6838269233703613
Node 284
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.018 )
( 11.773711001681953 8.770920307670977 0.31186382698487325 0.662622695789095 3.1196627616882324
Node 285
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.002 )
( 18.25171258123317 14.55123515206217 0.4231892822767824 0.47823313

( 12.319851399065454 10.090170419977051 0.42564232623906273 0.61467948340088 3.5395545959472656
Node 337
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.033 )
( 10.882952179900123 8.095293779727673 0.2971525377329409 0.6594951396318331 3.3470633029937744
Node 338
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.04 )
( 17.810186965579575 15.934731543003267 0.6814311749115393 0.4395487225466703 2.7137351036071777
Node 339
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.035 )
( 11.177531782334487 8.645947603417572 0.34501588706568426 0.6475507972494963 3.3001503944396973
Node 340
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.022 )
( 10.867627591836836 8.165801851944627 0.3287737042787937 0.6588375263814363 3.328066825866699
Node 341
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.035 )
( 11.907864265259489 9.297325440217898 0.3698338797745478 0.6227639802642597 2.6947991847991943
Node 342
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 31.402604697859633 29.90606630824372 1.0 0.0 0.007978439331054688
Node

( 11.048942569184998 7.831499309212943 0.3523252245123296 0.5735532327751807 2.9999513626098633
Node 392
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.005 )
( 10.97381589548644 7.740724034161851 0.3675130649802714 0.5773419379890201 3.4058992862701416
Node 393
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.025 )
( 12.037773269163429 9.217031684592186 0.4942963094428528 0.5300637068309357 3.051842212677002
Node 394
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.013 )
( 10.599670251084357 7.994894610203079 0.45891719349284243 0.5838455697053029 2.951165199279785
Node 395
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.029 )
( 11.863517048846138 8.732797209697962 0.4336633764871705 0.5344817758194098 3.4856631755828857
Node 396
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.02 )
( 10.675214406710998 8.087953857190438 0.4589600995158459 0.5753503734147258 3.273221254348755
Node 397
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.016 )
( 12.911172308372548 11.04577084048286 0.6899543654596739 0.48905985078

( 13.806813399415137 10.464170192573308 0.4426213340827569 0.4558058406870069 2.012590169906616
Node 446
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.015 )
( 11.172464879453894 8.85789773152318 0.4926689828946323 0.5585216192833073 2.7038238048553467
Node 447
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.026 )
( 10.121378912839116 7.766505759279386 0.4296685371419881 0.6033427667375358 2.7456116676330566
Node 448
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 25.361611403357447 23.446326164874545 1.0 0.0 0.009972810745239258
Node 449
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.054 )
( 10.680641359856034 7.939274310793419 0.3820560094649063 0.5805060542639389 3.8317811489105225
Node 450
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.026 )
( 9.801761368076608 7.225735339672095 0.36974399455541795 0.61653351704818 3.180508852005005
Node 451
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.018 )
( 11.30339906634337 9.056856277572951 0.4948604968214701 0.5606907180572358 2.7466602325439453
Node 452

( 10.750941558848362 8.106897937110974 0.3251259778462358 0.6502150791193397 2.4474942684173584
Node 501
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.007 )
( 10.305931862196841 7.4844109333002935 0.2870796688192426 0.6657389091966794 2.863344192504883
Node 502
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.01 )
( 11.840425469218593 8.710140447557471 0.31774527439388667 0.6203735782989167 2.370673418045044
Node 503
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 31.291392704345057 29.595391278375136 1.0 0.0 0.00796818733215332
Node 504
( (2286, 12) (2286, 6) (558, 12) (558, 6) 0.0 )
( 31.515428267388998 29.822909199522098 1.0 0.0 0.006982564926147461
Node 505
( (2286, 12) (2286, 6) (558, 12) (558, 6) -0.002 )
( 14.262003373221042 10.644684090895462 0.32271013352602457 0.5484167432802887 2.830436944961548
RMSE: 16.99109243197558, MAE: 14.339648089839203, MAPE: 0.5611229517844768, ACC: 0.4244349961548423
